In [ ]:
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from app import app

import math
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
import dash as dash
import json
from datetime import datetime as dt
from datetime import timedelta
import re

from modules import module_vobcfault
from modules import module_commLoss
from views.ViewTrainmoveClass import ViewTrainmoveClass
from views.view_commLossListClass import ViewCommLossListClass


import util as util
import multiprocessing as mp

In [101]:
def get_switch_filter_val( row, start_date = start_date, end_date = end_date):
    switchId = row["switchId"]
    Limit = row["amt"]
    start_date, end_date = util.date2str2(start_date,end_date)
    Limit = int(Limit)
    query = ("SELECT interval, switchId from dlr_switch_move"
            " where interval > 0 and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            " order by interval desc LIMIT {}").format(switchId, start_date, end_date, Limit)
    
    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["interval"].min()


In [ ]:
def get_switch_count(start_date, end_date):
    start_date, end_date = util.date2str2(start_date,end_date)

    query = ("SELECT switchId, COUNT(*) as amt from dlr_switch_move" 
            " where interval >= 0 and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )" 
            " and loggedAt >= '{}' and loggedAt < '{}'" 
            " group by switchId").format( start_date, end_date)
    
    L = util.run_query(query)

    return L

In [ ]:
start_date = dt(2012, 1, 1)
end_date = dt(2020, 1, 1)
start_date, end_date  = util.date2str2(start_date, end_date )

In [ ]:
pool = mp.Pool(mp.cpu_count())

In [ ]:
df1 = get_switch_count(start_date, end_date)
df1["amt"] = df1["amt"].div(1000)
df1[df1.index.duplicated()]

In [90]:

df2 = df1
df2['max'] = df2.apply(lambda row: get_switch_filter_val(start_date, end_date, row["switchId"], row["amt"]), axis=1)
df2

,switchId,amt,max
0,101,350.077,3.34
1,104,1.475,4.18
2,105,1.475,4.18
3,106,2.252,54.34
4,107,2.251,54.34
...,...,...,...
129,410,1.187,6.13
130,411,8.881,4.04
131,412,8.881,4.04
132,413,132.290,3.97


In [ ]:
import dask.dataframe as dd
from dask.multiprocessing import get

In [100]:
df2 = df1

pool = mp.Pool(mp.cpu_count())


results = [pool.apply(get_switch_filter_val, args = (row)) for row in df2]


pool.close()    
results

AttributeError: 'str' object has no attribute 'switchId'

In [89]:

data = df1
ddata = dd.from_pandas(df1, npartitions=1)
res = ddata.map_partitions(lambda df: df1.apply(lambda row: get_switch_filter_val(start_date, end_date, row["switchId"], row["amt"]), axis=1), meta=(None, "f8")).compute(scheduler='threads')
res

3.04 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
df2 = df1

data = df1
ddata = dd.from_pandas(df1, npartitions=6)
res = ddata.map_partitions(lambda df: df2.assign(max1 = get_switch_filter_val(start_date, end_date, df2["switchId"], df2["amt"])), meta=("f64")).compute(scheduler='threads')

res

TypeError: cannot convert the series to <class 'int'>